In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mtcnn

In [ ]:
import pandas
import mtcnn
from IPython.display import Image
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from tqdm import tqdm
import sklearn.metrics as metrics

# Making some random counts using pre defined algorithms.

In [ ]:
base_directory = r"/kaggle/input/count-the-number-of-faces-present-in-an-image"
train_directory = r"/train"
image_directory = base_directory + train_directory + r"/image_data"

train_dataframe = pandas.read_csv(base_directory + r"/train/train.csv")
test_dataframe = pandas.read_csv(base_directory + r"/test.csv")
target = train_dataframe.columns[-1]

train_dataframe.head(3)

# Checking sample images 

In [ ]:
def display_image(filename, head_count='Test set'):
    display(Image(image_directory + "/" + filename, width = 700, height = 200), "HeadCount: "+ head_count.astype(str))
    
display_image(train_dataframe.iloc[0, 0], train_dataframe.iloc[0, 1])

In [ ]:
def draw_image_with_boxes(filename, result_list):
    fig, ax = pyplot.subplots(figsize=(15, 8))
    # load all the images from the dataset.
    data = pyplot.imread(image_directory + "/" + filename)
    # plot the images
    pyplot.imshow(data)
    # get the context for drawing boxes
    ax = pyplot.gca()
    # plot each box
    for result in result_list:
        # get coordinates
        x, y, width, height = result['box']
        # create the shape
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        # draw the box
        ax.add_patch(rect)
        # draw the dots
        for key, value in result['keypoints'].items():
            # create and draw dot
            dot = Circle(value, radius=2, color='red')
            ax.add_patch(dot)
    # show the plot
    pyplot.show()

In [ ]:
errors = []

detector = MTCNN()
def detect_faces(filename):
    try:
        current_image = pyplot.imread(image_directory + "/" + filename)
        boxes = detector.detect_faces(current_image)
        length = len(boxes)
    except ValueError:
        errors.append(filename)
        return filename, [], 0
    
    return filename, boxes, length

## Model predictions and mapping on images . 

In [ ]:
def single_image_analysis(filename, expected_pred):
    _, detected_boxes, _ = detect_faces(train_dataframe.iloc[0, 0])
    draw_image_with_boxes(train_dataframe.iloc[0, 0], detected_boxes)

    print(f"expected prediction: {train_dataframe.iloc[0, 1]}, actually detected: {len(detected_boxes)}")
    

single_image_analysis(train_dataframe.iloc[0, 0], train_dataframe.iloc[0, 1])

### now the algorithm is predicting the human faces correctly.

In [ ]:
image_files = train_dataframe.iloc[0:50, 0]
targets = train_dataframe.iloc[0:50, 1]
predicted = []

for image in tqdm(image_files):
    predicted.append(detect_faces(image))

In [ ]:
predicted_faces = [pred[2] for pred in predicted]
print(f"r2 score: {metrics.r2_score(targets, predicted_faces)}")

incorrect_predictions = []
for itr in range(targets.shape[0]):
    if targets[itr] != predicted_faces[itr]:
        current_list = [] 
        current_list.append(train_dataframe.iloc[itr, :].values)
        current_list.append(predicted_faces[itr])
        
        incorrect_predictions.append(current_list)